# Checkers
I have decided to join the Board class and the Game class into one class. The way they have implemented the Board class makes it difficult to implement a checkers board. At the moment it just stores {coordinate, player}. In checkers we need to move players, delete players, promote players etc.. Of course we could modify it, but from an OOP point of view I think it is fine for the Checkers game to possess and control a board internally.

We start with two lists of pieces. The boolean next to the location (i,j) is the king status. If the piece is taken we replace its location with None. That is, the lists P[0] and P[1] will always have size 12.

In [17]:
# A standard game of checkers. 8x8 board.
class Checkers():
    def __init__(self):
        # initial positions
        W = []#[(x,y) for y in range(0,3) for x in range(y%2,8,2)]
        B = []#[(x,y) for y in range(5,8) for x in range(y%2,8,2)]
        W.append((2,4))
        B.append((3,3))
        # for debugging
        self.piece_count = 1
        self.P = {'W' : W, 'B' : B}
        # we record king status as a mapping from location to boolean
        W_King = {location : True for location in W}
        B_King = {location : True for location in B}
        self.Kings = {'W' : W_King,'B' : B_King}
        self.to_move = 'B'

    def remove_self_intersections(self, piece):
        # There are 4 possible moves.
        turn = self.to_move
        if(turn == 'W'):
            moves = []
            if(piece[1] < 7):
                if(piece[0] > 0):
                    moves.append((piece[0] - 1, piece[1] + 1))
                if(piece[0] < 7):
                    moves.append((piece[0] + 1, piece[1] + 1))
            if(self.Kings[turn][piece] and piece[1] > 0):
                if(piece[0] > 0):
                    moves.append((piece[0] - 1, piece[1] - 1))
                if(piece[0] < 7):
                    moves.append((piece[0] + 1, piece[1] - 1))
            available_moves = []
            for m in moves:
                if m in self.P['W']:
                    continue
                available_moves.append(m)
            return available_moves
            
        if(turn == 'B'):
            moves = []
            if(piece[1] > 0 ):
                if(piece[0] > 0):
                    moves.append((piece[0] - 1, piece[1] - 1))
                if(piece[0] < 7):
                    moves.append((piece[0] + 1, piece[1] - 1))
            if(self.Kings[turn][piece] and piece[1] < 0):
                if(piece[0] > 0):
                    moves.append((piece[0] - 1, piece[1] + 1))
                if(piece[0] < 7):
                    moves.append((piece[0] + 1, piece[1] + 1))
            available_moves = []
            for m in moves:
                if m in self.P['B']:
                    continue
                available_moves.append(m)
            return available_moves

    def capture_cycle(self, position, captured):
        directions = []
        p = (position[0] - 1, position[1] - 1)
        if p[0] in range(8) and p[1] in range(8): directions.append(p)
        p = (position[0] + 1, position[1] - 1)
        if p[0] in range(8) and p[1] in range(8): directions.append(p)
        p = (position[0] - 1, position[1] + 1)
        if p[0] in range(8) and p[1] in range(8): directions.append(p)
        p = (position[0] + 1, position[1] + 1)
        if p[0] in range(8) and p[1] in range(8): directions.append(p)
        available_directions = []
        for d in directions:
            # we don't want to recapture and we can not do anything
            # if one of our pieces is  in the way. 
            if d not in captured and d not in self.P[self.to_move]:
                available_directions.append(d)
        capture_directions = 0
        for d in available_directions:
            rival = 'B' if self.to_move == 'W' else 'W'
            if d in self.P[rival]:
                if d in captured: continue
                # we have a rival piece, now we need to see if we can jump it.
                direction = (d[0] - position[0], d[1] - position[1])
                destination = (d[0] + direction[0], d[1] + direction[1])
                if destination[0] not in range(8) or destination[1] not in range(8): continue
                # We also need to check whether the destination is open. That is, clear.
                if destination in self.P['W'] or destination in self.P['B']: continue
                # At this point we have a valid square to land on.
                # We need to add the piece to captured and call capture_cycle again
                captured.append(d)
                capture_directions += 1
                self.capture_cycle(destination, captured)
        if(len(captured) > 0):
            captured.append(None)
            captured.append(position)
            captured.append(None)
                 
    
    def generate_capture_actions(self, i, piece, moves):
        captured = []
        other_player = 'B' if self.to_move == 'W' else 'W'
        other_player_piece_locations = self.P[other_player]
        best_capture = None
        for move in moves:
            # can we take a piece?
            if move in other_player_piece_locations:
                # We need to check whether the destination is valid
                direction = (move[0] - piece[0], move[1] - piece[1])
                destination = (move[0] + direction[0], move[1] + direction[1])
                if destination[0] not in range(8) or destination[1] not in range(8): continue
                # We also need to check whether the destination is open. That is, clear.
                if destination in self.P['W'] or destination in self.P['B']: continue
                # At this point we have a valid square to land on.
                captures = [move]
                final_destination = None
                self.capture_cycle(destination, captures)
                captures = list(dict.fromkeys(captures))
                if best_capture == None:
                    best_capture = captures
                else:
                    if(len(captures) > len(best_capture)):
                        best_capture = captures
        return [i, best_capture]

    # action is a tuple:
    # (i, [taken_pieces, None, final_position], make_king)
    def actions(self):
        actions = []
        for i in range(self.piece_count):
            turn = self.to_move
            piece = self.P[turn][i]
            # If the piece is taken, just continue.
            if piece == None: continue
            moves = self.remove_self_intersections(piece)
            # If there are no moves ,just continue.
            if(len(moves) == 0): continue
            action = self.generate_capture_actions(i, piece, moves)
            actions.append(action)
        capture_only_actions = []
        for a in actions:
            if a[1] != None:
                capture_only_actions.append(a)
        if len(capture_only_actions) > 0:
            return capture_only_actions

        # if capture_only_actions is of size zero, 
        # we need to look into just moving pieces
        actions = []
        for i in range(self.piece_count):
            turn = self.to_move
            piece = self.P[turn][i]
            # If the piece is taken, just continue.
            if piece == None: continue
            moves = self.remove_self_intersections(piece)
            for move in moves:
                actions.append([i, [None, move]])

        return actions

In [18]:
c = Checkers()
actions = c.actions()
for a in actions:
    print(a)

[0, [None, (2, 2)]]
[0, [None, (4, 2)]]
